In [83]:
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv  

In [84]:
# Update path to match where your JAR file is downloaded - make sure it's absolute path
jdbc_jar_path = "/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/library/postgresql-42.7.3.jar"
spark = SparkSession.builder \
        .appName("PostgreSQLIngestion") \
        .config("spark.jars", jdbc_jar_path) \
        .config("spark.driver.extraClassPath", jdbc_jar_path) \
        .getOrCreate()

In [85]:
spark

In [86]:
print(spark.sparkContext._conf.get('spark.jars'))

/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/library/postgresql-42.7.3.jar


In [87]:
# Load environment variables
load_dotenv()

DB_PASSWORD = os.environ.get('DB_PASSWORD')
DB_NAME = os.environ.get('DB_NAME')
DB_USER = os.environ.get('DB_USER')
SCHEMA_NAME = os.environ.get('SCHEMA_NAME')
REGISTRATION_TABLE = os.environ.get('REGISTRATION_TABLE')
print(DB_PASSWORD, DB_NAME, DB_USER, SCHEMA_NAME, REGISTRATION_TABLE)

123abc456 sampledb riddhi_psql users_schema registration


In [88]:

#This sets up the metadata to ingest your source table.
# JDBC URL and properties for PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/"+ DB_NAME
jdbc_properties = {
    "user": DB_USER,
    "password": DB_PASSWORD,
    "driver": "org.postgresql.Driver"
}

table_name = f"{SCHEMA_NAME}.{REGISTRATION_TABLE}"
print(jdbc_url, jdbc_properties, table_name)

jdbc:postgresql://localhost:5432/sampledb {'user': 'riddhi_psql', 'password': '123abc456', 'driver': 'org.postgresql.Driver'} users_schema.registration


In [89]:
# Read data from PostgreSQL into a DataFrame
df = spark.read.jdbc( \
url=jdbc_url, \
table=table_name, \
properties=jdbc_properties \
) 

# Show the schema and first few rows of the DataFrame
df.printSchema()
df.show(1)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- password: string (nullable = true)
 |-- created_at: timestamp (nullable = true)

+---+----------+--------------------+----------+--------------------+
| id|      name|               email|  password|          created_at|
+---+----------+--------------------+----------+--------------------+
|  8|Ben Stokes|benstoke@example.com|ghhjahkjk!|2025-07-12 14:15:...|
+---+----------+--------------------+----------+--------------------+
only showing top 1 row


In [90]:
# Write the DataFrame to Parquet format
base_dir = "/home/riddhi/Documents/Riddhi-Tech/Projects/Python-Projects/CRUD-Flask/"
output_dir = os.path.join(base_dir, "datasets", "parquet", "registration_data")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
df.write.parquet(f"{output_dir}/output_data", mode="overwrite")

In [ ]:
df2 = spark.read.parquet(f"{output_dir}/output_data")
df2.show(8)

+---+--------------+--------------------+---------------+--------------------+
| id|          name|               email|       password|          created_at|
+---+--------------+--------------------+---------------+--------------------+
|  8|    Ben Stokes|benstoke@example.com|     ghhjahkjk!|2025-07-12 14:15:...|
|  9|    Benny Dahl|Benny.dahl@exampl...|      hfkhdfhk!|2025-07-14 15:20:...|
| 10|  Riddhi Patel|riddhipatel@examp...|hfkhhfdhshdfhk!|2025-07-14 17:15:...|
| 11|Bhumi Chudasma|   bhumi@example.com|hfkhhfdhshdfhk!|2025-07-14 17:26:...|
|  7|Jaimin Bhadani|Jaimin.bhadani@ex...|   hjkfdhskfhh!|2025-07-08 14:51:...|
+---+--------------+--------------------+---------------+--------------------+



25/07/15 10:57:02 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 43515028 ms exceeds timeout 120000 ms
25/07/15 10:57:02 WARN SparkContext: Killing executors is not supported by current scheduler.
25/07/15 10:57:02 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at